In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pywt

In [20]:
df = pd.read_excel("df_20.xlsx")
df.head()

,AvgAir_T_TempC,MaxAir_T_TempC,MinAir_T_TempC,Avg_Soil_TP20_TempC,Max_Soil_TP20_TempC,Min_Soil_TP20_TempC
0,8.16,11.96,1.40,8.530,8.85,8.230
1,5.48,7.33,3.47,7.926,8.28,7.774
2,6.41,9.25,1.81,8.130,8.47,7.927
3,9.17,10.67,5.04,8.790,9.15,8.440
4,3.38,5.71,-0.33,8.590,9.09,8.280


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038 entries, 0 to 1037
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   AvgAir_T_TempC       1038 non-null   float64
 1   MaxAir_T_TempC       1038 non-null   float64
 2   MinAir_T_TempC       1038 non-null   float64
 3   Avg_Soil_TP20_TempC  1038 non-null   float64
 4   Max_Soil_TP20_TempC  1038 non-null   float64
 5   Min_Soil_TP20_TempC  1038 non-null   float64
dtypes: float64(6)
memory usage: 48.8 KB


In [22]:
df["Avg_Soil_TP20_TempC_Plus_1"] = df["Avg_Soil_TP20_TempC"].shift(-1)
df["Avg_Soil_TP20_TempC_Plus_2"] = df["Avg_Soil_TP20_TempC"].shift(-2)

df["Max_Soil_TP20_TempC_Plus_1"] = df["Max_Soil_TP20_TempC"].shift(-1)
df["Max_Soil_TP20_TempC_Plus_2"] = df["Max_Soil_TP20_TempC"].shift(-2)

df["Min_Soil_TP20_TempC_Plus_1"] = df["Min_Soil_TP20_TempC"].shift(-1)
df["Min_Soil_TP20_TempC_Plus_2"] = df["Min_Soil_TP20_TempC"].shift(-2)

df["AvgAir_T_TempC_Minus_6"] = df["AvgAir_T_TempC"].shift(6)
df["AvgAir_T_TempC_Minus_5"] = df["AvgAir_T_TempC"].shift(5)
df["AvgAir_T_TempC_Minus_4"] = df["AvgAir_T_TempC"].shift(4)
df["AvgAir_T_TempC_Minus_3"] = df["AvgAir_T_TempC"].shift(3)
df["AvgAir_T_TempC_Minus_2"] = df["AvgAir_T_TempC"].shift(2)
df["AvgAir_T_TempC_Minus_1"] = df["AvgAir_T_TempC"].shift(1)

df["MaxAir_T_TempC_Minus_6"] = df["MaxAir_T_TempC"].shift(6)
df["MaxAir_T_TempC_Minus_5"] = df["MaxAir_T_TempC"].shift(5)
df["MaxAir_T_TempC_Minus_4"] = df["MaxAir_T_TempC"].shift(4)
df["MaxAir_T_TempC_Minus_3"] = df["MaxAir_T_TempC"].shift(3)
df["MaxAir_T_TempC_Minus_2"] = df["MaxAir_T_TempC"].shift(2)
df["MaxAir_T_TempC_Minus_1"] = df["MaxAir_T_TempC"].shift(1)


df["MinAir_T_TempC_Minus_6"] = df["MinAir_T_TempC"].shift(6)
df["MinAir_T_TempC_Minus_5"] = df["MinAir_T_TempC"].shift(5)
df["MinAir_T_TempC_Minus_4"] = df["MinAir_T_TempC"].shift(4)
df["MinAir_T_TempC_Minus_3"] = df["MinAir_T_TempC"].shift(3)
df["MinAir_T_TempC_Minus_2"] = df["MinAir_T_TempC"].shift(2)
df["MinAir_T_TempC_Minus_1"] = df["MinAir_T_TempC"].shift(1)

In [23]:
df = df[["AvgAir_T_TempC_Minus_6","MaxAir_T_TempC_Minus_6","MinAir_T_TempC_Minus_6",
          "AvgAir_T_TempC_Minus_5","MaxAir_T_TempC_Minus_5","MinAir_T_TempC_Minus_5",
          "AvgAir_T_TempC_Minus_4","MaxAir_T_TempC_Minus_4","MinAir_T_TempC_Minus_4",
          "AvgAir_T_TempC_Minus_3","MaxAir_T_TempC_Minus_3","MinAir_T_TempC_Minus_3",
          "AvgAir_T_TempC_Minus_2","MaxAir_T_TempC_Minus_2","MinAir_T_TempC_Minus_2",
          "AvgAir_T_TempC_Minus_1","MaxAir_T_TempC_Minus_1","MinAir_T_TempC_Minus_1",
          "AvgAir_T_TempC","MaxAir_T_TempC","MinAir_T_TempC",
          "Avg_Soil_TP20_TempC","Max_Soil_TP20_TempC","Min_Soil_TP20_TempC",
          "Avg_Soil_TP20_TempC_Plus_1","Max_Soil_TP20_TempC_Plus_1","Min_Soil_TP20_TempC_Plus_1",
          "Avg_Soil_TP20_TempC_Plus_2","Max_Soil_TP20_TempC_Plus_2","Min_Soil_TP20_TempC_Plus_2"]]
df.dropna(inplace=True)

In [24]:
df.reset_index(drop="True", inplace=True)

In [25]:
def WaveletTransform(x,walvelettype,level):
    
    size = np.shape(x)
    length = (size[0] // (2**level)) * (2 ** level)
    x = x.iloc[0:length]
    bigmats = []
    for i in x.columns:
        vec = x[i].values
        cAs_cDs = pywt.swt(vec, wavelet=walvelettype, level=level,axis=0) 
        for j in cAs_cDs:
            bigmats.append(j[0])
            bigmats.append(j[1])

    
    features_num = len(bigmats)
    x = np.zeros((length, features_num))
    for i in range(0 ,len(bigmats)):
        for j in range(0,len(bigmats[0])):
            x[j,i] = x[j,i] + bigmats[i][j]
    
    return(x)

In [26]:
x = df.drop(["Avg_Soil_TP20_TempC","Max_Soil_TP20_TempC","Min_Soil_TP20_TempC",
            "Avg_Soil_TP20_TempC_Plus_1","Max_Soil_TP20_TempC_Plus_1","Min_Soil_TP20_TempC_Plus_1",
            "Avg_Soil_TP20_TempC_Plus_2","Max_Soil_TP20_TempC_Plus_2","Min_Soil_TP20_TempC_Plus_2"],axis=1)

In [27]:
y = df[["Avg_Soil_TP20_TempC","Max_Soil_TP20_TempC","Min_Soil_TP20_TempC",
            "Avg_Soil_TP20_TempC_Plus_1","Max_Soil_TP20_TempC_Plus_1","Min_Soil_TP20_TempC_Plus_1",
            "Avg_Soil_TP20_TempC_Plus_2","Max_Soil_TP20_TempC_Plus_2","Min_Soil_TP20_TempC_Plus_2"]].values

In [28]:
x = WaveletTransform(x,"db5",5)

In [29]:
x.shape

(1024, 210)

In [30]:
y = df[0:x.shape[0]]

In [31]:
x = pd.DataFrame(x)

In [32]:
df = pd.concat([x,y],axis=1)

In [33]:
df

,0,1,2,3,4,5,6,7,8,9,...,MinAir_T_TempC,Avg_Soil_TP20_TempC,Max_Soil_TP20_TempC,Min_Soil_TP20_TempC,Avg_Soil_TP20_TempC_Plus_1,Max_Soil_TP20_TempC_Plus_1,Min_Soil_TP20_TempC_Plus_1,Avg_Soil_TP20_TempC_Plus_2,Max_Soil_TP20_TempC_Plus_2,Min_Soil_TP20_TempC_Plus_2
0,68.521901,17.125962,80.918016,18.499315,57.324270,1.505259,35.943367,0.287648,26.367510,-4.310982,...,4.16,7.685,7.827,7.598,10.930,25.420,7.666,7.693,8.630,7.440
1,69.864985,19.036618,80.356873,17.770988,58.293480,4.936171,35.485424,1.596280,25.789807,0.923497,...,2.22,10.930,25.420,7.666,7.693,8.630,7.440,7.224,7.451,6.960
2,71.233887,20.658360,79.803331,15.821483,59.116780,8.517321,36.649358,-1.295042,17.930719,2.165125,...,0.77,7.693,8.630,7.440,7.224,7.451,6.960,7.327,7.898,6.849
3,72.628403,22.015623,79.333324,13.158899,59.729453,9.575284,38.144801,-4.192752,10.285471,-0.609532,...,-0.23,7.224,7.451,6.960,7.327,7.898,6.849,7.777,8.470,7.312
4,74.037831,22.870567,78.953846,9.902946,60.200359,7.423347,38.420068,-1.774528,9.935624,-1.422230,...,-0.03,7.327,7.898,6.849,7.777,8.470,7.312,8.610,9.310,8.280
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,61.923002,4.258578,81.131265,3.279522,54.658989,-1.836325,48.413367,-7.408064,23.874455,0.530526,...,9.16,15.620,15.960,15.340,15.880,15.980,15.750,15.230,15.900,14.880
1020,63.275745,6.961267,81.494441,8.033681,54.776647,-0.704872,48.225825,-1.071688,24.252076,-3.412353,...,10.54,15.880,15.980,15.750,15.230,15.900,14.880,15.010,15.300,14.800
1021,64.585938,9.813100,81.679043,12.469320,55.021807,-0.797959,45.817422,5.591794,25.947943,1.267339,...,4.61,15.230,15.900,14.880,15.010,15.300,14.800,15.100,15.330,14.910
1022,65.881449,12.519510,81.644177,15.966304,55.480309,-1.446066,41.990818,3.298393,25.132408,1.920876,...,9.93,15.010,15.300,14.800,15.100,15.330,14.910,14.730,15.210,14.480


# 2 - Feature Selection

In [34]:
#df --> Dataframe
#Ys --> a list of all target variables
#y --> the main target variable - string
def FeatureSelection(df,Ys,y,corr):
    X = df.drop(Ys,axis=1)
    Y = df[y]
    df_final = pd.concat([X,Y],axis=1)
    indexes = df_final.corr()[df_final.corr()[y] > corr].index
    df_final = df_final[indexes]
    return df_final

In [35]:
Ys = ["Avg_Soil_TP20_TempC", "Max_Soil_TP20_TempC", "Min_Soil_TP20_TempC","Avg_Soil_TP20_TempC_Plus_1",
     "Max_Soil_TP20_TempC_Plus_1","Min_Soil_TP20_TempC_Plus_1", "Avg_Soil_TP20_TempC_Plus_2", 
      "Max_Soil_TP20_TempC_Plus_2", "Min_Soil_TP20_TempC_Plus_2"]

In [36]:
df_avg = FeatureSelection(df,Ys,y = "Avg_Soil_TP20_TempC",corr=0.85)
df_max = FeatureSelection(df,Ys,y = "Max_Soil_TP20_TempC",corr=0.85)
df_min = FeatureSelection(df,Ys,y = "Min_Soil_TP20_TempC",corr=0.85)
df_avg_plus1 = FeatureSelection(df,Ys,y = "Avg_Soil_TP20_TempC_Plus_1",corr=0.85)
df_max_plus1 = FeatureSelection(df,Ys,y = "Max_Soil_TP20_TempC_Plus_1",corr=0.85)
df_min_plus1 = FeatureSelection(df,Ys,y = "Min_Soil_TP20_TempC_Plus_1",corr=0.85)
df_avg_plus2 = FeatureSelection(df,Ys,y = "Avg_Soil_TP20_TempC_Plus_2",corr=0.85)
df_max_plus2 = FeatureSelection(df,Ys,y = "Max_Soil_TP20_TempC_Plus_2",corr=0.85)
df_min_plus2 = FeatureSelection(df,Ys,y = "Min_Soil_TP20_TempC_Plus_2",corr=0.85)

# 3 - SVR Support vector machines for regression Problem

### Train test split and data transform

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [40]:
#df: data frame
#y: target variable --> string
def SplitTransform(df,y,test_size, scaler):
    X = df.drop(y,axis=1).values
    y = df[y].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=101)
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    data = [X_train, X_test,y_train,y_test]
    return data

### Building the Model

In [42]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score

In [43]:
model = SVR(gamma="auto")

### Avg_Soil_TP20_TempC

In [45]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [46]:
Ys = {"Avg_Soil_TP20_TempC":df_avg, "Max_Soil_TP20_TempC":df_max, "Min_Soil_TP20_TempC":df_min,"Avg_Soil_TP20_TempC_Plus_1":df_avg_plus1,
     "Max_Soil_TP20_TempC_Plus_1":df_max_plus1,"Min_Soil_TP20_TempC_Plus_1":df_min_plus1, "Avg_Soil_TP20_TempC_Plus_2":df_avg_plus2, 
      "Max_Soil_TP20_TempC_Plus_2":df_max_plus2, "Min_Soil_TP20_TempC_Plus_2":df_min_plus2}

In [47]:
i = "Avg_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Avg_Soil_TP20_TempC',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9344861544064378,
 'MAE': 1.3668383381991092,
 'MSE': 4.5878330419095175}

### Max_Soil_TP20_TempC

In [49]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [50]:
i = "Max_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Max_Soil_TP20_TempC',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9332733313485714,
 'MAE': 1.3789279555649236,
 'MSE': 4.903852370135363}

### Min_Soil_TP20_TempC

In [52]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [53]:
i = "Min_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Min_Soil_TP20_TempC',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9436478621963095,
 'MAE': 1.2366570454307835,
 'MSE': 3.7504142225236725}

### Avg_Soil_TP20_TempC_Plus_1

In [55]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [56]:
i = "Avg_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Avg_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9331285013626065,
 'MAE': 1.3662819969715556,
 'MSE': 4.657782983745238}

### Max_Soil_TP20_TempC_Plus_1

In [58]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [59]:
i = "Max_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Max_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9212797555685031,
 'MAE': 1.4483877620461822,
 'MSE': 5.831708792106268}

### Min_Soil_TP20_TempC_Plus_1

In [61]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [62]:
i = "Min_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Min_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9413613348918652,
 'MAE': 1.230206756775102,
 'MSE': 3.8872848338942823}

### Avg_Soil_TP20_TempC_Plus_2

In [64]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [65]:
i = "Avg_Soil_TP20_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Avg_Soil_TP20_TempC_Plus_2',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9238980448492103,
 'MAE': 1.4702432338804234,
 'MSE': 5.304416280663346}

### Max_Soil_TP20_TempC_Plus_2

In [67]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [68]:
i = "Max_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Max_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9212797555685031,
 'MAE': 1.4483877620461822,
 'MSE': 5.831708792106268}

### Min_Soil_TP20_TempC_Plus_1

In [70]:
param_grid = {"kernel" : ["rbf"],
             "C" : [40]}
grid = GridSearchCV(model,param_grid, cv = 5)

In [71]:
i = "Min_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

{'Target Variable': 'Min_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'C': 40, 'kernel': 'rbf'},
 'R Squared': 0.9413613348918652,
 'MAE': 1.230206756775102,
 'MSE': 3.8872848338942823}

# 4 - XGBoost (Extreme Gradient Boosting Method) for regression problem

### Building the model

In [72]:
import xgboost as xg

In [73]:
model = xg.XGBRegressor(objective ='reg:squarederror') 

### Avg_Soil_TP20_TempC

In [74]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0],
    'reg_lambda': [1.0],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [75]:
i = "Avg_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.956, total=   1.1s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.950, total=   0.9s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.0s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.937, total=   0.9s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.951, total=   0.9s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.953, total=   0.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.6s finished


{'Target Variable': 'Avg_Soil_TP20_TempC',
 'Best Parameters': {'gamma': 0,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1.0},
 'R Squared': 0.9569656753216029,
 'MAE': 1.2154147876671384,
 'MSE': 3.0136270418418794}

### Max_Soil_TP20_TempC

In [76]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0],
    'reg_lambda': [1.0],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [77]:
i = "Max_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.957, total=   0.8s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.949, total=   0.8s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.6s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.923, total=   1.1s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.937, total=   1.0s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.960, total=   0.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.4s finished


{'Target Variable': 'Max_Soil_TP20_TempC',
 'Best Parameters': {'gamma': 0,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1.0},
 'R Squared': 0.9588048527474022,
 'MAE': 1.2275596298668292,
 'MSE': 3.0274989681865665}

### Min_Soil_TP20_TempC

In [78]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0.25],
    'reg_lambda': [1.0],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [79]:
i = "Min_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.965, total=   0.8s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.949, total=   1.0s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.7s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.947, total=   0.8s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.953, total=   0.7s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1.0, score=0.953, total=   1.0s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.2s finished


{'Target Variable': 'Min_Soil_TP20_TempC',
 'Best Parameters': {'gamma': 0.25,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1.0},
 'R Squared': 0.9605400150531102,
 'MAE': 1.1376846084532801,
 'MSE': 2.626187657350837}

### Avg_Soil_TP20_TempC_Plus_1

In [80]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [81]:
i = "Avg_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.961, total=   0.8s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.951, total=   0.9s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.7s remaining:    0.0s


[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.947, total=   0.8s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.942, total=   0.9s
[CV] gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=0, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.949, total=   0.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.1s finished


{'Target Variable': 'Avg_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'gamma': 0,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9538679125784265,
 'MAE': 1.2912807486645588,
 'MSE': 3.213226205113024}

### Max_Soil_TP20_TempC_Plus_1

In [82]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.1],
    'gamma': [1],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [83]:
i = "Max_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1, score=0.961, total=   0.8s
[CV] gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1, score=0.951, total=   0.8s
[CV] gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.6s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1, score=0.933, total=   1.2s
[CV] gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1, score=0.943, total=   1.0s
[CV] gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.1, max_depth=7, n_estimators=150, reg_lambda=1, score=0.954, total=   0.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.6s finished


{'Target Variable': 'Max_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'gamma': 1,
  'learning_rate': 0.1,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9431921722693682,
 'MAE': 1.3863300165928805,
 'MSE': 4.208405484887334}

### Min_Soil_TP20_TempC_Plus_1

In [84]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [0.25],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [85]:
i = "Min_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.968, total=   0.9s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.951, total=   0.8s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.6s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.955, total=   1.0s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.945, total=   0.7s
[CV] gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.962, total=   0.7s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.1s finished


{'Target Variable': 'Min_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'gamma': 0.25,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9571900225429272,
 'MAE': 1.1714680586681738,
 'MSE': 2.837966652231124}

### Avg_Soil_TP20_TempC_Plus_2

In [86]:
param_grid = {
    'max_depth': [7],
    'learning_rate': [0.05],
    'gamma': [1],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [87]:
i = "Avg_Soil_TP20_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.952, total=   0.8s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.934, total=   0.8s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.5s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.958, total=   0.8s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.944, total=   0.7s
[CV] gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=7, n_estimators=150, reg_lambda=1, score=0.944, total=   0.7s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.7s finished


{'Target Variable': 'Avg_Soil_TP20_TempC_Plus_2',
 'Best Parameters': {'gamma': 1,
  'learning_rate': 0.05,
  'max_depth': 7,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9474224215816702,
 'MAE': 1.3063670526247522,
 'MSE': 3.6647332175295455}

### Max_Soil_TP20_TempC_Plus_2

In [88]:
param_grid = {
    'max_depth': [4],
    'learning_rate': [0.05],
    'gamma': [0.25],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [89]:
i = "Max_Soil_TP20_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1, score=0.949, total=   0.7s
[CV] gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1, score=0.935, total=   0.4s
[CV] gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s


[CV]  gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1, score=0.961, total=   0.4s
[CV] gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1, score=0.950, total=   0.4s
[CV] gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1 
[CV]  gamma=0.25, learning_rate=0.05, max_depth=4, n_estimators=150, reg_lambda=1, score=0.939, total=   0.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.4s finished


{'Target Variable': 'Max_Soil_TP20_TempC_Plus_2',
 'Best Parameters': {'gamma': 0.25,
  'learning_rate': 0.05,
  'max_depth': 4,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9363758390621363,
 'MAE': 1.4243813476190939,
 'MSE': 4.677461086240653}

### Min_Soil_TP20_TempC_Plus_2

In [90]:
param_grid = {
    'max_depth': [6],
    'learning_rate': [0.05],
    'gamma': [1],
    'reg_lambda': [1],
    'n_estimators' : [150]
      }
grid = GridSearchCV(model,param_grid, cv = 5, verbose=3)

In [91]:
i = "Min_Soil_TP20_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train)
best_params = grid.best_params_
score = grid.score(X_test,y_test)
prediction = grid.predict(X_test)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictinary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictinary

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1, score=0.964, total=   0.7s
[CV] gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1, score=0.938, total=   0.7s
[CV] gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s


[CV]  gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1, score=0.960, total=   0.7s
[CV] gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1, score=0.950, total=   0.6s
[CV] gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1 
[CV]  gamma=1, learning_rate=0.05, max_depth=6, n_estimators=150, reg_lambda=1, score=0.946, total=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.3s finished


{'Target Variable': 'Min_Soil_TP20_TempC_Plus_2',
 'Best Parameters': {'gamma': 1,
  'learning_rate': 0.05,
  'max_depth': 6,
  'n_estimators': 150,
  'reg_lambda': 1},
 'R Squared': 0.9489917329345795,
 'MAE': 1.2554338946605657,
 'MSE': 3.380242099153296}

# 5 - ANN

### Building the Model

In [92]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

In [93]:
def create_model(neurons,hidden_layers, optimizer, activation):
    model = Sequential()
    
    # Add aninput layer
    model.add(Dense(X_train.shape[1], activation = activation))
    
    for i in range(hidden_layers):
    # Add one hidden layer
        model.add(Dense(neurons, activation=activation))
        
        
    # Add an output layer 
    model.add(Dense(1))
    
    # Compile model
    model.compile(optimizer = optimizer, loss="mse")
    
    return model

In [94]:
model = KerasRegressor(build_fn=create_model, verbose = 3)

### Avg_Soil_TP20_TempC

In [95]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["relu"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1, verbose=3)

In [96]:
i = "Avg_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.1min remaining:  1.6min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.3min finished


Epoch 1/400
6/6 [==============================] - 0s 6ms/step - loss: 76.7795
Epoch 2/400
6/6 [==============================] - 0s 6ms/step - loss: 20.2010
Epoch 3/400
6/6 [==============================] - 0s 5ms/step - loss: 14.0378
Epoch 4/400
6/6 [==============================] - 0s 7ms/step - loss: 9.1044
Epoch 5/400
6/6 [==============================] - 0s 6ms/step - loss: 7.6444
Epoch 6/400
6/6 [==============================] - 0s 5ms/step - loss: 7.5518
Epoch 7/400
6/6 [==============================] - 0s 5ms/step - loss: 6.5652
Epoch 8/400
6/6 [==============================] - 0s 6ms/step - loss: 6.1443
Epoch 9/400
6/6 [==============================] - 0s 7ms/step - loss: 5.9110
Epoch 10/400
6/6 [==============================] - 0s 6ms/step - loss: 5.4423
Epoch 11/400
6/6 [==============================] - 0s 6ms/step - loss: 5.2029
Epoch 12/400
6/6 [==============================] - 0s 6ms/step - loss: 4.7759
Epoch 13/400
6/6 [==============================] - 0s 5ms

6/6 [==============================] - 0s 6ms/step - loss: 0.3628
Epoch 105/400
6/6 [==============================] - 0s 6ms/step - loss: 0.3076
Epoch 106/400
6/6 [==============================] - 0s 5ms/step - loss: 0.2911
Epoch 107/400
6/6 [==============================] - 0s 6ms/step - loss: 0.2988
Epoch 108/400
6/6 [==============================] - 0s 6ms/step - loss: 0.3090
Epoch 109/400
6/6 [==============================] - 0s 6ms/step - loss: 0.3690
Epoch 110/400
6/6 [==============================] - 0s 5ms/step - loss: 0.3358
Epoch 111/400
6/6 [==============================] - 0s 5ms/step - loss: 0.2946
Epoch 112/400
6/6 [==============================] - 0s 6ms/step - loss: 0.2620
Epoch 113/400
6/6 [==============================] - 0s 6ms/step - loss: 0.2429
Epoch 114/400
6/6 [==============================] - 0s 6ms/step - loss: 0.2256
Epoch 115/400
6/6 [==============================] - 0s 6ms/step - loss: 0.2039
Epoch 116/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 7ms/step - loss: 0.1207
Epoch 207/400
6/6 [==============================] - 0s 6ms/step - loss: 0.1168
Epoch 208/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0939
Epoch 209/400
6/6 [==============================] - 0s 6ms/step - loss: 0.1066
Epoch 210/400
6/6 [==============================] - 0s 7ms/step - loss: 0.0939
Epoch 211/400
6/6 [==============================] - 0s 6ms/step - loss: 0.1112
Epoch 212/400
6/6 [==============================] - 0s 7ms/step - loss: 0.1370
Epoch 213/400
6/6 [==============================] - 0s 6ms/step - loss: 0.1533
Epoch 214/400
6/6 [==============================] - 0s 6ms/step - loss: 0.1489
Epoch 215/400
6/6 [==============================] - 0s 7ms/step - loss: 0.1191
Epoch 216/400
6/6 [==============================] - 0s 6ms/step - loss: 0.1786
Epoch 217/400
6/6 [==============================] - 0s 6ms/step - loss: 0.2182
Epoch 218/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 6ms/step - loss: 0.0628
Epoch 309/400
6/6 [==============================] - 0s 6ms/step - loss: 0.0624
Epoch 310/400
6/6 [==============================] - 0s 6ms/step - loss: 0.0806
Epoch 311/400
6/6 [==============================] - ETA: 0s - loss: 0.052 - 0s 6ms/step - loss: 0.0821
Epoch 312/400
6/6 [==============================] - 0s 7ms/step - loss: 0.0598
Epoch 313/400
6/6 [==============================] - 0s 6ms/step - loss: 0.0579
Epoch 314/400
6/6 [==============================] - 0s 7ms/step - loss: 0.0563
Epoch 315/400
6/6 [==============================] - 0s 6ms/step - loss: 0.0553
Epoch 316/400
6/6 [==============================] - 0s 7ms/step - loss: 0.0477
Epoch 317/400
6/6 [==============================] - 0s 6ms/step - loss: 0.0559
Epoch 318/400
6/6 [==============================] - 0s 6ms/step - loss: 0.0645
Epoch 319/400
6/6 [==============================] - 0s 7ms/step - loss: 0.0880
Epoch 320/400
6/6 [===========

{'Target Variable': 'Avg_Soil_TP20_TempC',
 'Best Parameters': {'activation': 'relu',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9497942704880004,
 'MAE': 1.2449908980817181,
 'MSE': 3.5882747536413486}

### Max_Soil_TP20_TempC

In [97]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [98]:
i = "Max_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
6/6 [==============================] - 0s 3ms/step - loss: 134.1710
Epoch 2/400
6/6 [==============================] - 0s 3ms/step - loss: 104.5266
Epoch 3/400
6/6 [==============================] - 0s 3ms/step - loss: 87.6087
Epoch 4/400
6/6 [==============================] - 0s 4ms/step - loss: 79.4464
Epoch 5/400
6/6 [==============================] - 0s 2ms/step - loss: 77.0857
Epoch 6/400
6/6 [==============================] - 0s 3ms/step - loss: 76.8075
Epoch 7/400
6/6 [==============================] - 0s 2ms/step - loss: 76.7353
Epoch 8/400
6/6 [==============================] - 0s 2ms/step - loss: 76.7001
Epoch 9/400
6/6 [==============================] - 0s 3ms/step - loss: 76.5260
Epoch 10/400
6/6 [==============================] - 0s 3ms/step - loss: 76.1793
Epoch 11/400
6/6 [==============================] - 0s 3ms/step - loss: 75.6631
Epoch 12/400
6/6 [==============================] - 0s 3ms/step - loss: 74.8239
Epoch 13/400
6/6 [=============================

6/6 [==============================] - 0s 3ms/step - loss: 4.4262
Epoch 105/400
6/6 [==============================] - 0s 2ms/step - loss: 4.4299
Epoch 106/400
6/6 [==============================] - 0s 3ms/step - loss: 4.5045
Epoch 107/400
6/6 [==============================] - 0s 3ms/step - loss: 4.5305
Epoch 108/400
6/6 [==============================] - 0s 2ms/step - loss: 4.4659
Epoch 109/400
6/6 [==============================] - 0s 2ms/step - loss: 4.5076
Epoch 110/400
6/6 [==============================] - 0s 2ms/step - loss: 4.4940
Epoch 111/400
6/6 [==============================] - 0s 3ms/step - loss: 4.4293
Epoch 112/400
6/6 [==============================] - 0s 3ms/step - loss: 4.4174
Epoch 113/400
6/6 [==============================] - 0s 3ms/step - loss: 4.4050
Epoch 114/400
6/6 [==============================] - 0s 3ms/step - loss: 4.5250
Epoch 115/400
6/6 [==============================] - 0s 2ms/step - loss: 4.3902
Epoch 116/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 3ms/step - loss: 3.4962
Epoch 207/400
6/6 [==============================] - 0s 2ms/step - loss: 3.5395
Epoch 208/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4902
Epoch 209/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4669
Epoch 210/400
6/6 [==============================] - 0s 2ms/step - loss: 3.4811
Epoch 211/400
6/6 [==============================] - 0s 3ms/step - loss: 3.5110
Epoch 212/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6980
Epoch 213/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6435
Epoch 214/400
6/6 [==============================] - 0s 3ms/step - loss: 3.5644
Epoch 215/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4654
Epoch 216/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4827
Epoch 217/400
6/6 [==============================] - 0s 2ms/step - loss: 3.5983
Epoch 218/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 3ms/step - loss: 3.0446
Epoch 309/400
6/6 [==============================] - 0s 3ms/step - loss: 3.1046
Epoch 310/400
6/6 [==============================] - 0s 3ms/step - loss: 3.1144
Epoch 311/400
6/6 [==============================] - 0s 3ms/step - loss: 3.0732
Epoch 312/400
6/6 [==============================] - 0s 2ms/step - loss: 2.9216
Epoch 313/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9082
Epoch 314/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9700
Epoch 315/400
6/6 [==============================] - 0s 3ms/step - loss: 3.0214
Epoch 316/400
6/6 [==============================] - 0s 3ms/step - loss: 3.0482
Epoch 317/400
6/6 [==============================] - 0s 3ms/step - loss: 3.0570
Epoch 318/400
6/6 [==============================] - 0s 3ms/step - loss: 3.0321
Epoch 319/400
6/6 [==============================] - ETA: 0s - loss: 2.950 - 0s 3ms/step - loss: 2.9974
Epoch 320/400
6/6 [===========

{'Target Variable': 'Max_Soil_TP20_TempC',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.9451210895671671,
 'MAE': 1.5538010170051417,
 'MSE': 4.331812634771092}

### Min_Soil_TP20_TempC

In [99]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [100]:
i = "Min_Soil_TP20_TempC"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
6/6 [==============================] - 0s 3ms/step - loss: 88.0149
Epoch 2/400
6/6 [==============================] - 0s 3ms/step - loss: 73.0783
Epoch 3/400
6/6 [==============================] - 0s 3ms/step - loss: 68.7552
Epoch 4/400
6/6 [==============================] - 0s 4ms/step - loss: 68.6605
Epoch 5/400
6/6 [==============================] - 0s 2ms/step - loss: 68.5061
Epoch 6/400
6/6 [==============================] - 0s 2ms/step - loss: 67.4452
Epoch 7/400
6/6 [==============================] - 0s 4ms/step - loss: 66.1862
Epoch 8/400
6/6 [==============================] - 0s 3ms/step - loss: 64.4962
Epoch 9/400
6/6 [==============================] - 0s 2ms/step - loss: 61.4211
Epoch 10/400
6/6 [==============================] - 0s 3ms/step - loss: 56.3955
Epoch 11/400
6/6 [==============================] - 0s 3ms/step - loss: 49.5312
Epoch 12/400
6/6 [==============================] - 0s 3ms/step - loss: 41.8622
Epoch 13/400
6/6 [==============================]

6/6 [==============================] - 0s 3ms/step - loss: 3.6591
Epoch 105/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6209
Epoch 106/400
6/6 [==============================] - 0s 3ms/step - loss: 3.7377
Epoch 107/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8186
Epoch 108/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6871
Epoch 109/400
6/6 [==============================] - 0s 3ms/step - loss: 3.7546
Epoch 110/400
6/6 [==============================] - 0s 3ms/step - loss: 3.7543
Epoch 111/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6578
Epoch 112/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6269
Epoch 113/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6274
Epoch 114/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6636
Epoch 115/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6385
Epoch 116/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 3ms/step - loss: 3.2678
Epoch 207/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4238
Epoch 208/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4170
Epoch 209/400
6/6 [==============================] - 0s 3ms/step - loss: 3.2812
Epoch 210/400
6/6 [==============================] - 0s 3ms/step - loss: 3.2771
Epoch 211/400
6/6 [==============================] - 0s 3ms/step - loss: 3.2269
Epoch 212/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4009
Epoch 213/400
6/6 [==============================] - 0s 3ms/step - loss: 3.5398
Epoch 214/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3087
Epoch 215/400
6/6 [==============================] - 0s 3ms/step - loss: 3.2777
Epoch 216/400
6/6 [==============================] - 0s 3ms/step - loss: 3.1864
Epoch 217/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3863
Epoch 218/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 3ms/step - loss: 2.8462
Epoch 309/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9903
Epoch 310/400
6/6 [==============================] - 0s 3ms/step - loss: 2.8494
Epoch 311/400
6/6 [==============================] - 0s 3ms/step - loss: 2.7555
Epoch 312/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9013
Epoch 313/400
6/6 [==============================] - 0s 3ms/step - loss: 2.7528
Epoch 314/400
6/6 [==============================] - 0s 3ms/step - loss: 2.7464
Epoch 315/400
6/6 [==============================] - 0s 3ms/step - loss: 2.7918
Epoch 316/400
6/6 [==============================] - 0s 3ms/step - loss: 2.8243
Epoch 317/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9564
Epoch 318/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9511
Epoch 319/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9108
Epoch 320/400
6/6 [==============================] - 0

{'Target Variable': 'Min_Soil_TP20_TempC',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.948441589073355,
 'MAE': 1.42246493703572,
 'MSE': 3.5268607426292498}

### Avg_Soil_TP20_TempC_Plus_1

In [101]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [6],
    'optimizer' : ['Adam'],
    "activation" : ["relu"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [102]:
i = "Avg_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
6/6 [==============================] - 0s 9ms/step - loss: 87.1764
Epoch 2/400
6/6 [==============================] - 0s 8ms/step - loss: 20.6560
Epoch 3/400
6/6 [==============================] - 0s 10ms/step - loss: 14.5707
Epoch 4/400
6/6 [==============================] - 0s 7ms/step - loss: 8.6698
Epoch 5/400
6/6 [==============================] - 0s 7ms/step - loss: 8.0937
Epoch 6/400
6/6 [==============================] - 0s 10ms/step - loss: 6.6212
Epoch 7/400
6/6 [==============================] - 0s 7ms/step - loss: 6.1153
Epoch 8/400
6/6 [==============================] - 0s 7ms/step - loss: 6.1308
Epoch 9/400
6/6 [==============================] - 0s 8ms/step - loss: 5.8103
Epoch 10/400
6/6 [==============================] - 0s 10ms/step - loss: 5.2713
Epoch 11/400
6/6 [==============================] - 0s 7ms/step - loss: 5.1419
Epoch 12/400
6/6 [==============================] - ETA: 0s - loss: 4.497 - 0s 8ms/step - loss: 4.8497
Epoch 13/400
6/6 [=============

6/6 [==============================] - 0s 8ms/step - loss: 0.1785
Epoch 105/400
6/6 [==============================] - 0s 8ms/step - loss: 0.2086
Epoch 106/400
6/6 [==============================] - 0s 8ms/step - loss: 0.2077
Epoch 107/400
6/6 [==============================] - 0s 9ms/step - loss: 0.3462
Epoch 108/400
6/6 [==============================] - 0s 8ms/step - loss: 0.4221
Epoch 109/400
6/6 [==============================] - 0s 8ms/step - loss: 0.5245
Epoch 110/400
6/6 [==============================] - 0s 8ms/step - loss: 0.3797
Epoch 111/400
6/6 [==============================] - 0s 10ms/step - loss: 0.3156
Epoch 112/400
6/6 [==============================] - 0s 8ms/step - loss: 0.2543
Epoch 113/400
6/6 [==============================] - 0s 8ms/step - loss: 0.2494
Epoch 114/400
6/6 [==============================] - 0s 8ms/step - loss: 0.3172
Epoch 115/400
6/6 [==============================] - 0s 8ms/step - loss: 0.2539
Epoch 116/400
6/6 [==============================] - 

Epoch 206/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0733
Epoch 207/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0661
Epoch 208/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0517
Epoch 209/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0500
Epoch 210/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0429
Epoch 211/400
6/6 [==============================] - 0s 10ms/step - loss: 0.0519
Epoch 212/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0804
Epoch 213/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0719
Epoch 214/400
6/6 [==============================] - 0s 10ms/step - loss: 0.1367
Epoch 215/400
6/6 [==============================] - 0s 7ms/step - loss: 0.3266
Epoch 216/400
6/6 [==============================] - 0s 9ms/step - loss: 0.4072
Epoch 217/400
6/6 [==============================] - 0s 8ms/step - loss: 0.1470
Epoch 218/400
6/6 [===================

6/6 [==============================] - 0s 8ms/step - loss: 0.2350
Epoch 309/400
6/6 [==============================] - 0s 9ms/step - loss: 0.1394
Epoch 310/400
6/6 [==============================] - 0s 9ms/step - loss: 0.1095
Epoch 311/400
6/6 [==============================] - 0s 8ms/step - loss: 0.1110
Epoch 312/400
6/6 [==============================] - 0s 8ms/step - loss: 0.1073
Epoch 313/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0928
Epoch 314/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0810
Epoch 315/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0595
Epoch 316/400
6/6 [==============================] - 0s 8ms/step - loss: 0.0425
Epoch 317/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0304
Epoch 318/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0254
Epoch 319/400
6/6 [==============================] - 0s 9ms/step - loss: 0.0227
Epoch 320/400
6/6 [==============================] - 0

{'Target Variable': 'Avg_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'activation': 'relu',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 6,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9578639648202004,
 'MAE': 1.1561822018164705,
 'MSE': 2.945344601297506}

### Max_Soil_TP20_TempC_Plus_1

In [103]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [104]:
i = "Max_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
6/6 [==============================] - 0s 7ms/step - loss: 101.5837
Epoch 2/400
6/6 [==============================] - 0s 6ms/step - loss: 78.3210
Epoch 3/400
6/6 [==============================] - 0s 7ms/step - loss: 81.6084
Epoch 4/400
6/6 [==============================] - 0s 6ms/step - loss: 78.2632
Epoch 5/400
6/6 [==============================] - 0s 8ms/step - loss: 76.3687
Epoch 6/400
6/6 [==============================] - 0s 8ms/step - loss: 76.3835
Epoch 7/400
6/6 [==============================] - 0s 6ms/step - loss: 73.8542
Epoch 8/400
6/6 [==============================] - 0s 6ms/step - loss: 69.0197
Epoch 9/400
6/6 [==============================] - 0s 6ms/step - loss: 58.2314
Epoch 10/400
6/6 [==============================] - 0s 6ms/step - loss: 44.4716
Epoch 11/400
6/6 [==============================] - 0s 6ms/step - loss: 33.5069
Epoch 12/400
6/6 [==============================] - 0s 6ms/step - loss: 26.8002
Epoch 13/400
6/6 [==============================

6/6 [==============================] - 0s 6ms/step - loss: 4.6977
Epoch 105/400
6/6 [==============================] - 0s 6ms/step - loss: 4.7863
Epoch 106/400
6/6 [==============================] - 0s 6ms/step - loss: 4.8431
Epoch 107/400
6/6 [==============================] - 0s 6ms/step - loss: 4.7502
Epoch 108/400
6/6 [==============================] - 0s 6ms/step - loss: 4.6970
Epoch 109/400
6/6 [==============================] - 0s 6ms/step - loss: 4.5475
Epoch 110/400
6/6 [==============================] - 0s 7ms/step - loss: 4.5861
Epoch 111/400
6/6 [==============================] - 0s 6ms/step - loss: 4.5610
Epoch 112/400
6/6 [==============================] - 0s 6ms/step - loss: 4.5821
Epoch 113/400
6/6 [==============================] - 0s 6ms/step - loss: 4.5407
Epoch 114/400
6/6 [==============================] - 0s 6ms/step - loss: 4.5372
Epoch 115/400
6/6 [==============================] - 0s 5ms/step - loss: 4.6328
Epoch 116/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 7ms/step - loss: 4.0867
Epoch 207/400
6/6 [==============================] - 0s 6ms/step - loss: 4.1777
Epoch 208/400
6/6 [==============================] - 0s 6ms/step - loss: 4.1173
Epoch 209/400
6/6 [==============================] - 0s 6ms/step - loss: 4.0628
Epoch 210/400
6/6 [==============================] - 0s 6ms/step - loss: 4.1075
Epoch 211/400
6/6 [==============================] - 0s 6ms/step - loss: 4.1114
Epoch 212/400
6/6 [==============================] - 0s 6ms/step - loss: 4.0631
Epoch 213/400
6/6 [==============================] - 0s 6ms/step - loss: 4.0942
Epoch 214/400
6/6 [==============================] - 0s 6ms/step - loss: 3.9573
Epoch 215/400
6/6 [==============================] - 0s 6ms/step - loss: 4.0615
Epoch 216/400
6/6 [==============================] - 0s 6ms/step - loss: 4.1182
Epoch 217/400
6/6 [==============================] - 0s 6ms/step - loss: 4.0352
Epoch 218/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 6ms/step - loss: 3.4157
Epoch 309/400
6/6 [==============================] - 0s 7ms/step - loss: 3.3842
Epoch 310/400
6/6 [==============================] - 0s 6ms/step - loss: 3.5126
Epoch 311/400
6/6 [==============================] - 0s 6ms/step - loss: 3.6899
Epoch 312/400
6/6 [==============================] - 0s 6ms/step - loss: 3.6313
Epoch 313/400
6/6 [==============================] - 0s 6ms/step - loss: 3.9443
Epoch 314/400
6/6 [==============================] - 0s 6ms/step - loss: 4.0479
Epoch 315/400
6/6 [==============================] - 0s 6ms/step - loss: 3.7084
Epoch 316/400
6/6 [==============================] - 0s 7ms/step - loss: 3.4147
Epoch 317/400
6/6 [==============================] - 0s 6ms/step - loss: 3.4370
Epoch 318/400
6/6 [==============================] - 0s 7ms/step - loss: 3.6004
Epoch 319/400
6/6 [==============================] - 0s 5ms/step - loss: 3.5095
Epoch 320/400
6/6 [==============================] - 0

{'Target Variable': 'Max_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9358761939169647,
 'MAE': 1.5861185775610147,
 'MSE': 4.876996629975124}

### Min_Soil_TP20_TempC_Plus_1

In [105]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [106]:
i = "Min_Soil_TP20_TempC_Plus_1"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
6/6 [==============================] - 0s 3ms/step - loss: 96.7090
Epoch 2/400
6/6 [==============================] - 0s 3ms/step - loss: 77.2848
Epoch 3/400
6/6 [==============================] - 0s 3ms/step - loss: 69.9121
Epoch 4/400
6/6 [==============================] - 0s 3ms/step - loss: 68.7533
Epoch 5/400
6/6 [==============================] - 0s 4ms/step - loss: 69.0304
Epoch 6/400
6/6 [==============================] - 0s 3ms/step - loss: 68.8741
Epoch 7/400
6/6 [==============================] - 0s 2ms/step - loss: 68.0314
Epoch 8/400
6/6 [==============================] - 0s 4ms/step - loss: 66.8862
Epoch 9/400
6/6 [==============================] - 0s 3ms/step - loss: 65.4025
Epoch 10/400
6/6 [==============================] - 0s 3ms/step - loss: 63.0884
Epoch 11/400
6/6 [==============================] - 0s 3ms/step - loss: 59.2600
Epoch 12/400
6/6 [==============================] - 0s 3ms/step - loss: 53.9448
Epoch 13/400
6/6 [==============================]

6/6 [==============================] - 0s 3ms/step - loss: 3.6216
Epoch 105/400
6/6 [==============================] - 0s 3ms/step - loss: 3.5959
Epoch 106/400
6/6 [==============================] - 0s 4ms/step - loss: 3.6518
Epoch 107/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6342
Epoch 108/400
6/6 [==============================] - 0s 4ms/step - loss: 3.6102
Epoch 109/400
6/6 [==============================] - 0s 3ms/step - loss: 3.5758
Epoch 110/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6059
Epoch 111/400
6/6 [==============================] - 0s 3ms/step - loss: 3.5747
Epoch 112/400
6/6 [==============================] - 0s 3ms/step - loss: 3.5850
Epoch 113/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6268
Epoch 114/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6739
Epoch 115/400
6/6 [==============================] - 0s 3ms/step - loss: 3.5393
Epoch 116/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 3ms/step - loss: 3.1923
Epoch 207/400
6/6 [==============================] - 0s 3ms/step - loss: 3.2216
Epoch 208/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4927
Epoch 209/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4531
Epoch 210/400
6/6 [==============================] - 0s 3ms/step - loss: 3.2258
Epoch 211/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3747
Epoch 212/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4650
Epoch 213/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3061
Epoch 214/400
6/6 [==============================] - 0s 3ms/step - loss: 3.1781
Epoch 215/400
6/6 [==============================] - 0s 3ms/step - loss: 3.1436
Epoch 216/400
6/6 [==============================] - 0s 3ms/step - loss: 3.1792
Epoch 217/400
6/6 [==============================] - 0s 3ms/step - loss: 3.2095
Epoch 218/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 3ms/step - loss: 2.7119
Epoch 309/400
6/6 [==============================] - 0s 3ms/step - loss: 2.6532
Epoch 310/400
6/6 [==============================] - 0s 3ms/step - loss: 2.6682
Epoch 311/400
6/6 [==============================] - 0s 3ms/step - loss: 2.7344
Epoch 312/400
6/6 [==============================] - 0s 3ms/step - loss: 2.7194
Epoch 313/400
6/6 [==============================] - 0s 3ms/step - loss: 2.7568
Epoch 314/400
6/6 [==============================] - 0s 3ms/step - loss: 2.6911
Epoch 315/400
6/6 [==============================] - 0s 3ms/step - loss: 2.6329
Epoch 316/400
6/6 [==============================] - 0s 3ms/step - loss: 2.6819
Epoch 317/400
6/6 [==============================] - 0s 3ms/step - loss: 2.6484
Epoch 318/400
6/6 [==============================] - 0s 3ms/step - loss: 2.7827
Epoch 319/400
6/6 [==============================] - 0s 3ms/step - loss: 2.6496
Epoch 320/400
6/6 [==============================] - 0

{'Target Variable': 'Min_Soil_TP20_TempC_Plus_1',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.9480557874823472,
 'MAE': 1.345448732174062,
 'MSE': 3.4480541158711753}

### Avg_Soil_TP20_TempC_Plus_2

In [107]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [350],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [108]:
i = "Avg_Soil_TP20_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
6/6 [==============================] - 0s 6ms/step - loss: 97.4874
Epoch 2/400
6/6 [==============================] - 0s 6ms/step - loss: 74.1615
Epoch 3/400
6/6 [==============================] - 0s 7ms/step - loss: 77.2954
Epoch 4/400
6/6 [==============================] - 0s 8ms/step - loss: 73.6383
Epoch 5/400
6/6 [==============================] - 0s 6ms/step - loss: 72.5886
Epoch 6/400
6/6 [==============================] - 0s 6ms/step - loss: 72.4482
Epoch 7/400
6/6 [==============================] - 0s 7ms/step - loss: 70.3033
Epoch 8/400
6/6 [==============================] - 0s 6ms/step - loss: 65.3396
Epoch 9/400
6/6 [==============================] - 0s 6ms/step - loss: 53.1060
Epoch 10/400
6/6 [==============================] - 0s 6ms/step - loss: 37.0135
Epoch 11/400
6/6 [==============================] - 0s 6ms/step - loss: 26.6351
Epoch 12/400
6/6 [==============================] - 0s 6ms/step - loss: 21.5639
Epoch 13/400
6/6 [==============================]

6/6 [==============================] - 0s 7ms/step - loss: 4.3891
Epoch 105/400
6/6 [==============================] - 0s 7ms/step - loss: 4.3650
Epoch 106/400
6/6 [==============================] - 0s 7ms/step - loss: 4.4994
Epoch 107/400
6/6 [==============================] - 0s 6ms/step - loss: 4.5165
Epoch 108/400
6/6 [==============================] - 0s 7ms/step - loss: 4.6598
Epoch 109/400
6/6 [==============================] - 0s 7ms/step - loss: 4.5811
Epoch 110/400
6/6 [==============================] - 0s 6ms/step - loss: 4.5247
Epoch 111/400
6/6 [==============================] - 0s 6ms/step - loss: 4.3030
Epoch 112/400
6/6 [==============================] - 0s 6ms/step - loss: 4.5819
Epoch 113/400
6/6 [==============================] - 0s 6ms/step - loss: 4.3847
Epoch 114/400
6/6 [==============================] - 0s 7ms/step - loss: 4.3909
Epoch 115/400
6/6 [==============================] - 0s 6ms/step - loss: 4.4096
Epoch 116/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 6ms/step - loss: 4.1878
Epoch 207/400
6/6 [==============================] - 0s 5ms/step - loss: 4.2579
Epoch 208/400
6/6 [==============================] - 0s 6ms/step - loss: 4.4929
Epoch 209/400
6/6 [==============================] - 0s 6ms/step - loss: 4.0325
Epoch 210/400
6/6 [==============================] - 0s 6ms/step - loss: 4.1485
Epoch 211/400
6/6 [==============================] - 0s 5ms/step - loss: 4.1048
Epoch 212/400
6/6 [==============================] - 0s 5ms/step - loss: 4.0689
Epoch 213/400
6/6 [==============================] - 0s 5ms/step - loss: 4.2080
Epoch 214/400
6/6 [==============================] - 0s 6ms/step - loss: 4.2288
Epoch 215/400
6/6 [==============================] - 0s 6ms/step - loss: 4.2580
Epoch 216/400
6/6 [==============================] - 0s 6ms/step - loss: 4.0830
Epoch 217/400
6/6 [==============================] - 0s 5ms/step - loss: 4.0693
Epoch 218/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 6ms/step - loss: 3.5638
Epoch 308/400
6/6 [==============================] - 0s 7ms/step - loss: 3.4765
Epoch 309/400
6/6 [==============================] - 0s 7ms/step - loss: 3.4231
Epoch 310/400
6/6 [==============================] - 0s 8ms/step - loss: 3.4954
Epoch 311/400
6/6 [==============================] - 0s 6ms/step - loss: 3.4566
Epoch 312/400
6/6 [==============================] - 0s 7ms/step - loss: 3.5210
Epoch 313/400
6/6 [==============================] - 0s 7ms/step - loss: 3.3858
Epoch 314/400
6/6 [==============================] - 0s 5ms/step - loss: 3.3868
Epoch 315/400
6/6 [==============================] - 0s 6ms/step - loss: 3.4247
Epoch 316/400
6/6 [==============================] - 0s 6ms/step - loss: 3.5893
Epoch 317/400
6/6 [==============================] - 0s 6ms/step - loss: 3.7635
Epoch 318/400
6/6 [==============================] - 0s 6ms/step - loss: 3.4561
Epoch 319/400
6/6 [==============================] - 0

{'Target Variable': 'Avg_Soil_TP20_TempC_Plus_2',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 350,
  'optimizer': 'Adam'},
 'R Squared': 0.9334100461842665,
 'MAE': 1.5889304320923014,
 'MSE': 4.6495623239570705}

### Max_Soil_TP20_TempC_Plus_2

In [109]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [110]:
i = "Max_Soil_TP20_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
6/6 [==============================] - 0s 3ms/step - loss: 134.8646
Epoch 2/400
6/6 [==============================] - 0s 3ms/step - loss: 106.1197
Epoch 3/400
6/6 [==============================] - 0s 2ms/step - loss: 88.6253
Epoch 4/400
6/6 [==============================] - 0s 3ms/step - loss: 80.0245
Epoch 5/400
6/6 [==============================] - 0s 4ms/step - loss: 77.4101
Epoch 6/400
6/6 [==============================] - 0s 2ms/step - loss: 76.4157
Epoch 7/400
6/6 [==============================] - 0s 3ms/step - loss: 76.4464
Epoch 8/400
6/6 [==============================] - 0s 4ms/step - loss: 76.3417
Epoch 9/400
6/6 [==============================] - 0s 3ms/step - loss: 76.1986
Epoch 10/400
6/6 [==============================] - 0s 2ms/step - loss: 75.8926
Epoch 11/400
6/6 [==============================] - 0s 3ms/step - loss: 75.4283
Epoch 12/400
6/6 [==============================] - 0s 3ms/step - loss: 74.7012
Epoch 13/400
6/6 [=============================

Epoch 104/400
6/6 [==============================] - 0s 2ms/step - loss: 4.5072
Epoch 105/400
6/6 [==============================] - 0s 3ms/step - loss: 4.5719
Epoch 106/400
6/6 [==============================] - 0s 3ms/step - loss: 4.4873
Epoch 107/400
6/6 [==============================] - 0s 2ms/step - loss: 4.7260
Epoch 108/400
6/6 [==============================] - 0s 4ms/step - loss: 4.7052
Epoch 109/400
6/6 [==============================] - 0s 3ms/step - loss: 4.7244
Epoch 110/400
6/6 [==============================] - 0s 2ms/step - loss: 4.4950
Epoch 111/400
6/6 [==============================] - 0s 3ms/step - loss: 4.4707
Epoch 112/400
6/6 [==============================] - 0s 4ms/step - loss: 4.5605
Epoch 113/400
6/6 [==============================] - 0s 4ms/step - loss: 4.6201
Epoch 114/400
6/6 [==============================] - 0s 4ms/step - loss: 4.5166
Epoch 115/400
6/6 [==============================] - 0s 3ms/step - loss: 4.4443
Epoch 116/400
6/6 [=====================

6/6 [==============================] - 0s 3ms/step - loss: 3.8187
Epoch 207/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6128
Epoch 208/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6799
Epoch 209/400
6/6 [==============================] - 0s 3ms/step - loss: 3.7171
Epoch 210/400
6/6 [==============================] - 0s 3ms/step - loss: 3.7785
Epoch 211/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6158
Epoch 212/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6550
Epoch 213/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6107
Epoch 214/400
6/6 [==============================] - 0s 3ms/step - loss: 3.7301
Epoch 215/400
6/6 [==============================] - ETA: 0s - loss: 2.947 - 0s 3ms/step - loss: 3.7326
Epoch 216/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8330
Epoch 217/400
6/6 [==============================] - 0s 3ms/step - loss: 3.6164
Epoch 218/400
6/6 [===========

6/6 [==============================] - 0s 3ms/step - loss: 3.2106
Epoch 309/400
6/6 [==============================] - 0s 3ms/step - loss: 3.0469
Epoch 310/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9144
Epoch 311/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9534
Epoch 312/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9403
Epoch 313/400
6/6 [==============================] - ETA: 0s - loss: 2.632 - 0s 3ms/step - loss: 3.0163
Epoch 314/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9718
Epoch 315/400
6/6 [==============================] - 0s 4ms/step - loss: 2.9543
Epoch 316/400
6/6 [==============================] - 0s 3ms/step - loss: 3.1408
Epoch 317/400
6/6 [==============================] - 0s 3ms/step - loss: 3.2982
Epoch 318/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3424
Epoch 319/400
6/6 [==============================] - 0s 3ms/step - loss: 3.1563
Epoch 320/400
6/6 [===========

{'Target Variable': 'Max_Soil_TP20_TempC_Plus_2',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.9363544920491739,
 'MAE': 1.5715079106152445,
 'MSE': 4.760751333780444}

### Min_Soil_TP20_TempC_Plus_2

In [111]:
param_grid = {
    'batch_size': [128],
    'epochs': [400],
    'neurons': [150],
    'hidden_layers': [4],
    'optimizer' : ['Adam'],
    "activation" : ["sigmoid"]
      }
grid = GridSearchCV(model,param_grid,n_jobs=-1)

In [112]:
i = "Min_Soil_TP20_TempC_Plus_2"
X_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[0]
X_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[1]
y_train = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[2]
y_test = SplitTransform(df = Ys[i], y = i, test_size= 0.3, scaler = StandardScaler())[3]
grid.fit(X_train, y_train, verbose=True)
best_params = grid.best_params_
prediction = grid.predict(X_test)
score = explained_variance_score(y_test,prediction)
MSE = mean_squared_error(y_test,prediction)
MAE = mean_absolute_error(y_test,prediction) 
dictionary = {"Target Variable" : i, "Best Parameters" : best_params, "R Squared" : score, "MAE": MAE,
                "MSE": MSE}
dictionary

Epoch 1/400
6/6 [==============================] - 0s 3ms/step - loss: 89.8935
Epoch 2/400
6/6 [==============================] - 0s 3ms/step - loss: 74.8934
Epoch 3/400
6/6 [==============================] - 0s 3ms/step - loss: 69.3280
Epoch 4/400
6/6 [==============================] - 0s 4ms/step - loss: 68.6747
Epoch 5/400
6/6 [==============================] - 0s 3ms/step - loss: 69.1031
Epoch 6/400
6/6 [==============================] - 0s 3ms/step - loss: 68.4112
Epoch 7/400
6/6 [==============================] - 0s 3ms/step - loss: 67.2920
Epoch 8/400
6/6 [==============================] - 0s 3ms/step - loss: 65.8720
Epoch 9/400
6/6 [==============================] - 0s 3ms/step - loss: 63.6625
Epoch 10/400
6/6 [==============================] - 0s 3ms/step - loss: 60.1366
Epoch 11/400
6/6 [==============================] - 0s 3ms/step - loss: 54.8629
Epoch 12/400
6/6 [==============================] - 0s 3ms/step - loss: 48.3465
Epoch 13/400
6/6 [==============================]

6/6 [==============================] - 0s 2ms/step - loss: 3.8166
Epoch 105/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8598
Epoch 106/400
6/6 [==============================] - 0s 2ms/step - loss: 3.8748
Epoch 107/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8144
Epoch 108/400
6/6 [==============================] - 0s 3ms/step - loss: 3.9190
Epoch 109/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8806
Epoch 110/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8481
Epoch 111/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8156
Epoch 112/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8001
Epoch 113/400
6/6 [==============================] - 0s 3ms/step - loss: 3.7501
Epoch 114/400
6/6 [==============================] - 0s 3ms/step - loss: 3.7777
Epoch 115/400
6/6 [==============================] - 0s 3ms/step - loss: 3.8239
Epoch 116/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 3ms/step - loss: 3.4678
Epoch 207/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3882
Epoch 208/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3590
Epoch 209/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3129
Epoch 210/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3560
Epoch 211/400
6/6 [==============================] - 0s 2ms/step - loss: 3.2990
Epoch 212/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4062
Epoch 213/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4028
Epoch 214/400
6/6 [==============================] - 0s 3ms/step - loss: 3.3313
Epoch 215/400
6/6 [==============================] - 0s 2ms/step - loss: 3.5044
Epoch 216/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4118
Epoch 217/400
6/6 [==============================] - 0s 3ms/step - loss: 3.4169
Epoch 218/400
6/6 [==============================] - 0

6/6 [==============================] - 0s 3ms/step - loss: 2.8778
Epoch 309/400
6/6 [==============================] - 0s 4ms/step - loss: 2.8866
Epoch 310/400
6/6 [==============================] - 0s 3ms/step - loss: 2.8283
Epoch 311/400
6/6 [==============================] - 0s 3ms/step - loss: 2.8738
Epoch 312/400
6/6 [==============================] - 0s 3ms/step - loss: 2.9181
Epoch 313/400
6/6 [==============================] - 0s 2ms/step - loss: 2.7993
Epoch 314/400
6/6 [==============================] - 0s 2ms/step - loss: 2.7889
Epoch 315/400
6/6 [==============================] - 0s 2ms/step - loss: 2.8041
Epoch 316/400
6/6 [==============================] - 0s 3ms/step - loss: 2.8007
Epoch 317/400
6/6 [==============================] - ETA: 0s - loss: 1.966 - 0s 3ms/step - loss: 2.8886
Epoch 318/400
6/6 [==============================] - 0s 2ms/step - loss: 2.8849
Epoch 319/400
6/6 [==============================] - 0s 2ms/step - loss: 2.7538
Epoch 320/400
6/6 [===========

{'Target Variable': 'Min_Soil_TP20_TempC_Plus_2',
 'Best Parameters': {'activation': 'sigmoid',
  'batch_size': 128,
  'epochs': 400,
  'hidden_layers': 4,
  'neurons': 150,
  'optimizer': 'Adam'},
 'R Squared': 0.9415426468770167,
 'MAE': 1.4850655941312962,
 'MSE': 4.246319100125642}